In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install stanza

import stanza

corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

!ls $CORENLP_HOME
from stanza.server import CoreNLPClient

!pip install datasets

import datasets
import transformers

!pip install datasets rouge_score
import logging
import os

import re
import sys
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm


from datasets import load_dataset, load_metric
from dataclasses import dataclass, field
from typing import Optional

from rouge_score import rouge_scorer


from tqdm.notebook import tqdm

from stanza.server import CoreNLPClient
!pip install googletrans==4.0.0-rc1
import pandas as pd
import googletrans
from googletrans import Translator
import random
import spacy
spacy = spacy.load("en")




     |████████████████████████████████| 3.1 MB 11.5 MB/s 
     |████████████████████████████████| 596 kB 39.5 MB/s 
     |████████████████████████████████| 895 kB 68.5 MB/s 
     |████████████████████████████████| 61 kB 541 kB/s 
     |████████████████████████████████| 3.3 MB 44.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 78 kB 5.8 MB/s 
     |████████████████████████████████| 1.2 MB 31.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=1d805a50ac8646dddfcef39c3555f229264494496328d2379c64ea7b678827ca
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
     |████████████████████████████████| 432 kB 26.2 MB/s 
     |████████████████████████████████| 170 kB 59.8 MB/s 
  Cr

2021-12-04 05:35:25 INFO: Installing CoreNLP package into ./corenlp...


2021-12-04 05:35:36 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=./corenlp`.


build.xml				  jollyday.jar
corenlp.sh				  LIBRARY-LICENSES
CoreNLP-to-HTML.xsl			  LICENSE.txt
ejml-core-0.39.jar			  Makefile
ejml-core-0.39-sources.jar		  patterns
ejml-ddense-0.39.jar			  pom-java-11.xml
ejml-ddense-0.39-sources.jar		  pom-java-17.xml
ejml-simple-0.39.jar			  pom.xml
ejml-simple-0.39-sources.jar		  protobuf-java-3.11.4.jar
input.txt				  README.txt
input.txt.out				  RESOURCE-LICENSES
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.3.2.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.3.2-javadoc.jar
javax.json.jar				  stanford-corenlp-4.3.2-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.3.2-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.java
jaxb-impl-2.4.0-b180830.0438.jar	  StanfordDependenciesManual.p

In [ ]:
from datasets import load_dataset
xsum_train = load_dataset('xsum', split='train')
# xsum_train = pd.DataFrame(xsum_train)
# xsum_train.to_csv('xsum_train.csv')
# xsum_train.rename(columns={'document' : 'text'})

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [ ]:
#Filter XSUM DATASET
from spacy.lang.en.stop_words import STOP_WORDS
entities_to_track = ['PERSON', 'FAC', 'GPE', 'ORG', 'NORP', 'LOC', 'EVENT']

"""
https://github.com/amazon-research/fact-check-summarization
"""

def entity_match(ent, source):
    # split entity and match non-stop words
    ent_split = ent.split()
    result = []
    for l in range(len(ent_split), 1, -1):
        for start_i in range(len(ent_split) - l + 1):
            sub_ent = " ".join(ent_split[start_i:start_i+l])
            if re.search(re.escape(sub_ent), source, re.IGNORECASE):
                result.append(sub_ent)

        if result:
            break

    if result:
        return result

    else:
        for token in ent_split:
            if token.lower() not in STOP_WORDS or token == "US":
                if re.search(re.escape(token), source, re.IGNORECASE):
                    result.append(token)
        return result

    return []

def select_example(nlp, intro, abstract, filter_level=2):
    doc = nlp(abstract)
    en_count_in_summary = 0
    select = True
    for e in doc.ents:
        if e[0].ent_type_ in entities_to_track:
            en_count_in_summary += 1
            match_result = entity_match(e.text, intro, 2)
            if not match_result:
                select = False
                break
    # if select and en_count_in_summary>0:
    if select:
        return True
    else:
        return False

from datasets import load_dataset
xsum_train = load_dataset('xsum', split='train')

def check_entity_agreement(example):
  disagree = select_example(spacy, example['summary'], example['document'])
  example['entity_issue'] = disagree
  return example

xsum_train_filtered = xsum_train.map(check_entity_agreement, num_proc=8)

In [ ]:
def check_entity_agreement(example):
  disagree = select_example(spacy, example['summary'], example['document'])
  example['entity_issue'] = disagree
  return example

xsum_train_filtered = xsum_train.map(check_entity_agreement, num_proc=8)

In [ ]:
def align_ws(old_token, new_token):
    # Align trailing whitespaces between tokens
    if old_token[-1] == new_token[-1] == " ":
        return new_token
    elif old_token[-1] == " ":
        return new_token + " "
    elif new_token[-1] == " ":
        return new_token[:-1]
    else:
        return new_token


In [ ]:
NEGATABLE_TOKENS = ("are", "is", "was", "were", "have", "has", "had",
                                   "do", "does", "did", "can", "ca", "could", "may",
                                   "might", "must", "shall", "should", "will", "would")



def negation(summary):
  candidate_tokens = [token for token in summary if token.text in NEGATABLE_TOKENS]

  if not candidate_tokens:
      return None, None

  # choose random token to negate
  negated_token = random.choice(candidate_tokens)
  negated_index = negated_token.i
  L = len(summary)

  # negation occurs at the first negatable token (e.g. does not have)
  if negated_index > 0:
    if summary[negated_index-1].text in NEGATABLE_TOKENS:
      negated_token = summary[negated_index-1]
      negated_index = negated_index-1
  
  #check whether qualified by a negative
  is_negative = False
  if (L-1) > negated_index:
    if summary[negated_index + 1].text in ["not", "n't"]:
      is_negative = True
    elif summary[negated_index+1].text == "no":
      return None, None

  #add not/n't if is_negative is False, remove if True
  tokens = [token.text_with_ws for token in summary]
  if is_negative:
      if summary[negated_index + 1].text.lower() == "n't":
          if summary[negated_index + 1].text.lower() == "ca":
              tokens[negated_index] = "can" if tokens[negated_index].islower() else "Can"
          tokens[negated_index] = tokens[negated_index] + " "
      tokens.pop(negated_index + 1)

  else:
      if summary[negated_index].text.lower() in ["am", "may", "might", "must", "shall", "will"]:
          negation = "not "
      else:
          negation = random.choice(["not ", "n't "])

      if negation == "n't ":
          if summary[negated_index].text.lower() == "can":
              tokens[negated_index] = "ca" if tokens[negated_index].islower() else "Ca"
          else:
              tokens[negated_index] = tokens[negated_index][:-1]
      tokens.insert(negated_index + 1, negation)
  
  new_summary = spacy("".join(tokens))
  augmentation_span = [(negated_index, negated_index if is_negative else negated_index + 1)]

  if new_summary.text == summary.text:
    return None, None
  else:
    return new_summary, augmentation_span

  







In [ ]:
CLASS_TO_PRONOUN = {
            "SUBJECT": ["you", "he", "she", "we", "they"],
            "OBJECT": ["me", "you", "him", "her", "us", "them"],
            "POSSESSIVE": ["my", "your", "his", "her", "its", "our", "your", "their"],
            "REFLEXIVE": ["myself", "yourself", "himself", "itself", "ourselves", "yourselves", "themselves"]
        }


PRONOUN_TO_CLASS = {pronoun: key for (key, values) in CLASS_TO_PRONOUN.items() for pronoun in values}
PRONOUNS = [pronoun for pronoun in PRONOUN_TO_CLASS.keys()]


def pronounswap(summary):
  summary_pronouns = [token for token in summary if token.text.lower() in PRONOUNS]

  if not summary_pronouns:
    return None, None

  swap = random.choice(summary_pronouns)
  swap_index = swap.i
  swap_class = PRONOUN_TO_CLASS[swap.text.lower()]

  candidate_tokens = [token for token in CLASS_TO_PRONOUN[swap_class] if token != swap.text.lower()]

  if not candidate_tokens:
    return None, None

  swapped_token = random.choice(candidate_tokens)
  swapped_token = align_ws(swap.text_with_ws, swapped_token)
  swapped_token = swapped_token if swap.text.islower() else swapped_token.capitalize()

  summary_tokens = [token.text_with_ws for token in summary]
  summary_tokens[swap_index] = swapped_token

  new_summary = spacy("".join(summary_tokens))

  augmentation_span = [(swap_index, swap_index)]

  if new_summary.text == summary.text:
    return None, None

  else:
    return new_summary, augmentation_span







In [ ]:
ENTITY_CATEGORIES = ("PERSON", "ORG", "NORP", "FAC", "GPE", "LOC", "PRODUCT",
                           "WORK_OF_ART", "EVENT")

def entityswap(summary, source):
  source_ents = [ent for ent in source.ents if ent.label_ in ENTITY_CATEGORIES]
  summary_ents = [ent for ent in summary.ents if ent.label_ in ENTITY_CATEGORIES]

  if not source_ents or not summary_ents:
    return None, None

  swap = random.choice(summary_ents)
  candidate_ents = [ent for ent in source_ents if ent.text != swap.text and ent.text not in swap.text and swap.text not in ent.text]

  if not candidate_ents:
    return None, None

  swapped_ent = random.choice(candidate_ents)
  summary_tokens = [token.text_with_ws for token in summary]
  swapped_token = align_ws(swap.text_with_ws, swapped_ent.text_with_ws)
  summary_tokens = summary_tokens[:swap.start] + [swapped_token] + summary_tokens[swap.end:]

  new_summary = spacy("".join(summary_tokens))
  augmentation_span = [(swap.start, swap.start + len(swapped_ent) - 1)]

  if new_summary.text == summary.text:
    return None, None

  else:
    return new_summary, augmentation_span

In [ ]:
NUMBER_CATEGORIES = ("PERCENT", "MONEY", "QUANTITY", "CARDINAL")

def numberswap(summary, source):
  source_ents = [ent for ent in source.ents if ent.label_ in NUMBER_CATEGORIES]
  summary_ents = [ent for ent in summary.ents if ent.label_ in NUMBER_CATEGORIES]

  if not source_ents or not summary_ents:
    return None, None

  swap = random.choice(summary_ents)
  candidate_ents = [ent for ent in source_ents if ent.text != swap.text and ent.text not in swap.text and swap.text not in ent.text]

  if not candidate_ents:
    return None, None

  swapped_ent = random.choice(candidate_ents)
  summary_tokens = [token.text_with_ws for token in summary]
  swapped_token = align_ws(swap.text_with_ws, swapped_ent.text_with_ws)
  summary_tokens = summary_tokens[:swap.start] + [swapped_token] + summary_tokens[swap.end:]

  new_summary = spacy("".join(summary_tokens))
  augmentation_span = [(swap.start, swap.start + len(swapped_ent) - 1)]

  if new_summary.text == summary.text:
    return None, None

  else:
    return new_summary, augmentation_span


In [ ]:
DATE_CATEGORIES = ("DATE", "TIME")

def dateswap(summary, source):
  source_ents = [ent for ent in source.ents if ent.label_ in DATE_CATEGORIES]
  summary_ents = [ent for ent in summary.ents if ent.label_ in DATE_CATEGORIES]

  if not source_ents or not summary_ents:
    return None, None

  swap = random.choice(summary_ents)
  candidate_ents = [ent for ent in source_ents if ent.text != swap.text and ent.text not in swap.text and swap.text not in ent.text]

  if not candidate_ents:
    return None, None

  swapped_ent = random.choice(candidate_ents)
  summary_tokens = [token.text_with_ws for token in summary]
  swapped_token = align_ws(swap.text_with_ws, swapped_ent.text_with_ws)
  summary_tokens = summary_tokens[:swap.start] + [swapped_token] + summary_tokens[swap.end:]

  new_summary = spacy("".join(summary_tokens))
  augmentation_span = [(swap.start, swap.start + len(swapped_ent) - 1)]

  if new_summary.text == summary.text:
    return None, None

  else:
    return new_summary, augmentation_span

In [ ]:
from stanza.server import CoreNLPClient


POS_TAGS = ("NOUN", "ADJ", "PROPN")


def openIE_filter(extract, summary):
  new_text = ""
  extract = spacy(extract)

  start_index = 0
  for token in summary:
    if token.text != extract[0].text:
      start_index+=1
    else:
      break
  tracking_index = start_index
  for word in extract:
    while (summary[tracking_index].text != word.text):
      tracking_index+=1
    if summary[tracking_index].pos_ in POS_TAGS:
      break
  start_index = tracking_index

  while (summary[tracking_index].pos_ != "NOUN" and summary[tracking_index].pos_ != "PROPN"):
    new_text+=(summary[tracking_index].text_with_ws)
    tracking_index += 1
  while (summary[tracking_index].pos_ == "NOUN" or summary[tracking_index].pos_ == "PROPN"):
    new_text+=(summary[tracking_index].text_with_ws)
    tracking_index += 1
  return new_text, start_index, tracking_index
  


def s_o_swap(summary):
  summary_ents = [ent for ent in summary.ents]
  client = CoreNLPClient(timeout=150000000, be_quiet=True, annotators=['openie'], 
  endpoint='http://localhost:9002')
  client.start()
  import time
  time.sleep(2)
  story_triples = []

  document = client.annotate(summary.text, output_format='json')
  triples = []
  for sentence in document['sentences']:
      for triple in sentence['openie']:
          triples.append({
            'subject': triple['subject'],
            'relation': triple['relation'],
              'object': triple['object']
          })
  
  client.stop()
  time.sleep(2)

  if not triples:
    return None, None
  candidate_triples = []
  for triple in triples:
    subj = triple['subject']
    obj = triple['object']
    allow_triple = False
    for ent in summary_ents:
      if subj == ent.text or subj in ent.text or ent.text in subj:
        allow_triple = True
      if obj == ent.text or obj in ent.text or ent.text in obj:
        allow_triple = True
    if allow_triple == True:
      candidate_triples.append(triple)
  if not candidate_triples:
    return None, None
  triple_swap = random.choice(candidate_triples)
  triple_swap["subject"], s_start, s_end = openIE_filter(triple_swap["subject"], summary)
  triple_swap["object"], o_start, o_end = openIE_filter(triple_swap["object"], summary)
  if s_start == 0:
    triple_swap["object"] = triple_swap["object"].capitalize()
    if summary[s_start].pos_ != "PROPN":
      triple_swap["subject"] = triple_swap["subject"][0].lower() + triple_swap["subject"][1:]
  if o_start == 0:
    triple_swap["subject"] = triple_swap["subject"].capitalize()
    if summary[o_start].pos_ != "PROPN":
      triple_swap["object"] = triple_swap["object"][0].lower() + triple_swap["object"][1:]
  
  if s_end <= o_start:
    new_summary = summary[:s_start].text_with_ws + triple_swap["object"] + summary[s_end:o_start].text_with_ws + triple_swap["subject"] + summary[o_end:].text_with_ws
    augmentation_span = [(s_start, o_end - o_start + s_start-1), (o_end - s_end + s_start, o_end-1)]
  elif o_end <= s_start:
    new_summary = summary[:o_start].text_with_ws + triple_swap["subject"] + summary[o_end:s_start].text_with_ws + triple_swap["object"] + summary[s_end:].text_with_ws
    augmentation_span = [(o_start, s_end - s_start + o_start-1), (s_end - o_end + o_start-1, s_end-2)]
  else:
    return None, None
  new_summary = spacy(new_summary)
  if new_summary.text == summary.text:
    return None, None

  else:
    return new_summary, augmentation_span

In [ ]:
SOURCE_LANG = "en"
ACCEPTED_LANGS = ["fr", "de", "zh-TW", "es", "ru"]
translator = Translator()

def backtranslate(summary):
  new_lang = random.choice(ACCEPTED_LANGS)
  summary_trans = translator.translate(summary.text, dest=new_lang)
  summary_btrans = translator.translate(summary_trans.text, dest=SOURCE_LANG)

  new_summary = spacy(summary_btrans.text)
  augmentation_span = (new_summary[0].i, new_summary[-1].i)

  if summary.text == new_summary.text:
    return None, None
  else:
    return new_summary, [augmentation_span]


In [ ]:
NOISE_PROB = 0.05
DELETE_PROB = 0.8


def addnoise(summary, augmentation_span):
        summary_tokens = [token.text_with_ws for token in summary]

        new_summary = []
        for ix, token in enumerate(summary_tokens):
            # don't modify text inside an augmented span
            apply_augmentation = True
            if augmentation_span:
              for aug_span in augmentation_span:
                if aug_span:
                  span_start, span_end = aug_span
                  if span_start <= ix <= span_end:
                      apply_augmentation = False

            # decide whether to add noise
            if apply_augmentation and random.random() < NOISE_PROB:
                # decide whether to replicate or delete token
                if random.random() < DELETE_PROB:
                    # update spans and skip token
                    if augmentation_span:
                      for el in range(0, len(augmentation_span)):
                        aug_span = augmentation_span[el]
                        if aug_span:
                          span_start, span_end = aug_span
                          if ix < span_start:
                            span_start -= 1
                            span_end -= 1
                          aug_span = span_start, span_end
                          augmentation_span[el] = aug_span
                      if len(new_summary) > 0:
                        if new_summary[-1][-1] != " ":
                          new_summary[-1] = new_summary[-1] + " "
                      continue      
                else:
                  if augmentation_span:
                    for el in range(0, len(augmentation_span)):
                      aug_span = augmentation_span[el]
                      if aug_span:
                        span_start, span_end = aug_span
                        if ix < span_start:
                          span_start += 1
                          span_end += 1
                        aug_span = span_start, span_end
                        augmentation_span[el] = aug_span

                  new_summary.append(token)
            new_summary.append(token)
        new_summary = spacy("".join(new_summary))

        if summary.text == new_summary.text:
            return None
        else:
            return new_summary

In [ ]:
new_summaries = []
perturbations = ["bt", "prn", "dat", "num", "ent", "neg"]
changed_summaries = {}
AUGMENT_PROB = 0.5

for i in range(0, 10):
  summary = xsum_train["summary"][i]
  article = xsum_train["document"][i]
  summary = spacy(summary)
  article = spacy(article)
  ags = []
  new_summary = summary
  if random.random() < AUGMENT_PROB:
    pert_type = random.choice(perturbations)
    if pert_type == "bt":
      new_summary, ags = backtranslate(summary)
    if pert_type == "prn":
      new_summary, ags = pronounswap(summary)
    if pert_type == "dat":
      new_summary, ags = dateswap(summary, article)
    if pert_type == "num":
      new_summary, ags = numberswap(summary, article)
    if pert_type == "ent":
      new_summary, ags = entityswap(summary, article)
    if pert_type == "neg":
      new_summary, ags = negation(summary)
    if not new_summary and not ags:
      new_summary = summary
      ags = []
    new_summary = addnoise(new_summary, ags)
    
    if not new_summary:
      new_summary = summary
    print(i, pert_type)
    changed_summaries[i] = new_summary.text
  new_summaries.append(new_summary.text)


1 dat
4 bt
6 ent
8 ent
9 neg


In [ ]:
10 % 10

In [ ]:
# Choose 40%

xsum_train_corrupted = xsum_train
corrupted_summaries = xsum_train_corrupted['summary']

corrupt_percent = 0.4

corrupt_indicies = np.random.choice(range(len(xsum_train_corrupted)), size=int(corrupt_percent*len(xsum_train_corrupted)))
corruption_groups = np.split(corrupt_indicies, len(perturbations))

total_corruption_count = 0
for pert_type, indicies in zip(perturbations, corruption_groups):
  for index in indicies:
    if pert_type == "bt":
      new_summary, ags = backtranslate(summary)
    if pert_type == "prn":
      new_summary, ags = pronounswap(summary)
    if pert_type == "dat":
      new_summary, ags = dateswap(summary, article)
    if pert_type == "num":
      new_summary, ags = numberswap(summary, article)
    if pert_type == "ent":
      new_summary, ags = entityswap(summary, article)
    if pert_type == "neg":
      new_summary, ags = negation(summary)
    if not new_summary and not ags:
      new_summary = summary
      ags = []
    new_summary = addnoise(new_summary, ags)
    if new_summary:
      corrupted_summaries[index] = new_summary.text
      total_corruption_count += 1

AttributeError: ignored

In [ ]:
xsum_train_corrupted['corrupt_summary'] = corrupted_summaries

In [ ]:
xsum_train_corrupted.add_column('corrupt_summary', corrupted_summaries)

Dataset(features: {'document': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}, num_rows: 204017)

In [ ]:
ags = []
if not ags:
  print("A")

In [ ]:
new_summaries = []
perturbations = ["bt", "prn", "dat", "num", "ent", "neg"]
changed_summaries = {}
AUGMENT_PROB = 0.3

for i in range(7,8):
  summary = xsum_train["summary"][i]
  article = xsum_train["document"][i]
  summary = spacy(summary)
  article = spacy(article)
  ags = []
  new_summary = summary
  for pert_type in perturbations:
    if pert_type == "bt":
      new_summary, ags = backtranslate(summary)
    if pert_type == "prn":
      new_summary, ags = pronounswap(summary)
    if pert_type == "dat":
      new_summary, ags = dateswap(summary, article)
    if pert_type == "num":
      new_summary, ags = numberswap(summary, article)
    if pert_type == "ent":
      new_summary, ags = entityswap(summary, article)
    if pert_type == "neg":
      new_summary, ags = negation(summary)
    # if not new_summary and not ags:
      # new_summary = summary
      # ags = []
    # new_summary = addnoise(new_summary, ags)
    
    # if not new_summary:
      # new_summary = summary
    print(i, pert_type)
    changed_summaries[pert_type] = new_summary

  changed_summaries['original'] = summary.text
  new_summaries.append(new_summary)


In [ ]:
changed_summaries

In [ ]:
s_o_swap(spacy(xsum_train["summary"][10]))

In [ ]:
xsum_train["summary"][10]

In [ ]:
#s_o_swap
#added_predicateswap

def predicateswap(summary, source):
  source_verbs = [token for token in source if token.pos_ == "VERB"]
  source_adverbs = [token for token in source if token.pos_ == "ADV"]
  summary_preds = [token for token in summary if token.pos_ == "VERB" or token.pos_ == "ADV"]
  if not summary_preds or not source_adverbs or not source_verbs:
    return None, None
  original = random.choice(summary_preds)
  ix = original.i
  if original.pos_ == "VERB":
    new = random.choice(source_verbs)
  else:
    new = random.choice(source_adverbs)

  summary_tokens = [token.text_with_ws for token in summary]
  summary_tokens[ix] = new.text_with_ws
  new_summary = "".join(summary_tokens)
  new_summary = spacy(new_summary)
  augmentation_span = (ix, ix)
  if new_summary.text == summary.text:
    return None, None
  else:
    return new_summary, augmentation_span

In [ ]:
#smart swap picks out a relation and uses howNet to perturb it (unfortunately it takes a while)

!pip install anytree
!pip install OpenHowNet
!pip install googletrans==4.0.0-rc1
import pandas as pd
import googletrans
from googletrans import Translator
translator = Translator()
import random


import OpenHowNet


OpenHowNet.download()









In [ ]:
hownet_dict_advanced = OpenHowNet.HowNetDict(use_sim=True)

In [ ]:
def sem_pert(word, N):
  translated_word = translator.translate(word, dest='zh-cn')
  translated_word = translated_word.text
  for i in range(0, N):
    query_result = hownet_dict_advanced.get_nearest_words_via_sememes(translated_word,15)
    if not query_result:
      return None
    translated_word = random.choice((random.choice(query_result))['synset'])['word']
  replacement_word = translator.translate(translated_word, dest='en')
  if replacement_word == translated_word:
    return None
  return replacement_word.text